In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn as sk
import pickle

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf

2025-09-04 22:07:28.912855: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-04 22:07:29.115161: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-04 22:07:29.282179: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757023649.444135 2430978 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757023649.491942 2430978 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757023649.826314 2430978 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
columns = ["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"]

len(columns)

14

In [3]:
data = pd.read_csv('heart.csv')
data.columns = columns
data.dropna(inplace=True)
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,67,0,3,115,564,0,2,160,0,1.6,2,0,7,1
1,57,1,2,124,261,0,0,141,0,0.3,1,0,7,-1
2,64,1,4,128,263,0,0,105,1,0.2,2,1,7,1
3,74,0,2,120,269,0,2,121,1,0.2,1,1,3,1
4,65,1,4,120,177,0,0,140,0,0.4,1,0,7,1


In [5]:
#changes object types to numeric
le = sk.preprocessing.LabelEncoder()
for col in data.select_dtypes(include=['object']).columns:
    data[col] = le.fit_transform(data[col])

print(data.head())

   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   67    0   3       115   564    0        2      160      0      1.6      2   
1   57    1   2       124   261    0        0      141      0      0.3      1   
2   64    1   4       128   263    0        0      105      1      0.2      2   
3   74    0   2       120   269    0        2      121      1      0.2      1   
4   65    1   4       120   177    0        0      140      0      0.4      1   

   ca  thal  num  
0   0     7    1  
1   0     7   -1  
2   1     7    1  
3   1     3    1  
4   0     7    1  


In [6]:
X_columns = columns[:-1]
Y_columns = columns[-1]
print(X_columns, Y_columns)

# print(data.drop([Y_columns], axis=1).columns)
# print(data[X_columns].columns)

X = data.drop([Y_columns], axis=1)
Y = data[Y_columns]

['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal'] num


In [7]:
X_train, x_test, Y_train, y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=42)

# Then scale
scaler = sk.preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)  # learn scaling from training set
x_test = scaler.transform(x_test)

mean = scaler.mean_
std = np.sqrt(scaler.var_)

In [8]:
print(len(data), len(X_train))

269 215


In [8]:
print(X_train.shape)
print(Y_train.shape)
Y_train = (Y_train == 1).astype(int)   # converts -1 → 0, 1 → 1
y_test  = (y_test == 1).astype(int)


(215, 13)
(215,)


In [9]:
model = tf.keras.models.Sequential([
  tf.keras.Input(shape=(13,)),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

2025-09-04 22:10:38.080095: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
model.compile(optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy'])

In [11]:
Y_train[0]

np.int64(1)

In [12]:
X_train[0].shape

(13,)

In [17]:
model.fit(X_train, Y_train, epochs=5)
test_loss, test_acc = model.evaluate(x_test, y_test)
print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)
model.summary()

Epoch 1/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6279 - loss: 0.6705 
Epoch 2/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6837 - loss: 0.6201 
Epoch 3/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6605 - loss: 0.6061 
Epoch 4/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6233 - loss: 0.6342 
Epoch 5/5
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6884 - loss: 0.6220 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7593 - loss: 0.5497

Test accuracy: 0.7592592835426331

Test loss: 0.5497412085533142


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │            68 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 893 (3.49 KB)

 Trainable params: 297 (1.16 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 596 (2.33 KB)

In [45]:
print(x_test.shape)

(54, 13)


In [19]:
predictions = model.predict(x_test)
for idx, item  in enumerate(predictions):
  # print("Predicted: ", item, "Original: ", y_test[idx])
  # pred = model.predict(x_test[0:1])  # probability vector
  predicted_class = item  # index of the largest probability
  print("Predicted class:", predicted_class[0] > 0.5, "Original: ", y_test.iloc[idx] == 1)
  # print("Predicted Probabilities: ", item)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Predicted class: False Original:  True
Predicted class: False Original:  False
Predicted class: False Original:  False
Predicted class: True Original:  True
Predicted class: False Original:  False
Predicted class: True Original:  True
Predicted class: True Original:  False
Predicted class: False Original:  False
Predicted class: True Original:  True
Predicted class: False Original:  False
Predicted class: True Original:  True
Predicted class: True Original:  True
Predicted class: False Original:  False
Predicted class: True Original:  True
Predicted class: True Original:  True
Predicted class: False Original:  False
Predicted class: True Original:  True
Predicted class: True Original:  True
Predicted class: True Original:  False
Predicted class: False Original:  False
Predicted class: True Original:  False
Predicted class: True Original:  True
Predicted class: True Original:  False
Predicted class: True Original:  True
Predicted class: True Origina